In [185]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format 

O objetivo desse notebook é processar e preparar os dados para o treino

In [186]:
#Leitura dos dados

data_pagamentos = pd.read_parquet('../data/intermediate/data_pagamentos.parquet', engine='fastparquet')
data_cadastro = pd.read_parquet('../data/intermediate/data_cadastro.parquet', engine='fastparquet')
data_hist = pd.read_parquet('../data/intermediate/data_hist.parquet', engine='fastparquet')
data_objetivo = pd.read_parquet('../data/intermediate/data_objetivo.parquet', engine='fastparquet')

In [187]:
data_pagamentos.columns


Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
       'DATA_VENCIMENTO', 'VALOR_A_PAGAR', 'TAXA', 'DIAS_ATE_PAGAMENTO',
       'INADIMPLENTE', 'DIAS_DE_ATRASO', 'PRAZO_PAGAMENTO_DIAS'],
      dtype='object')

In [194]:

data_cadastro['FLAG_PF'] = (data_cadastro['FLAG_PF'] == 'PF').astype(int)
data_cadastro.columns
#PJ = 0 e PF = 1

Index(['ID_CLIENTE', 'DATA_CADASTRO', 'DDD', 'FLAG_PF', 'SEGMENTO_INDUSTRIAL',
       'DOMINIO_EMAIL', 'PORTE', 'CEP_2_DIG'],
      dtype='object')

In [193]:
data_hist.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS'], dtype='object')

In [192]:
data_objetivo.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_VENCIMENTO',
       'VALOR_A_PAGAR', 'TAXA'],
      dtype='object')

In [191]:
## Criando features historicas

data_pagamentos.isnull().sum()

ID_CLIENTE                0
SAFRA_REF                 0
DATA_EMISSAO_DOCUMENTO    0
DATA_PAGAMENTO            0
DATA_VENCIMENTO           0
VALOR_A_PAGAR             0
TAXA                      0
DIAS_ATE_PAGAMENTO        0
INADIMPLENTE              0
DIAS_DE_ATRASO            0
PRAZO_PAGAMENTO_DIAS      0
dtype: int64

In [ ]:
#Como não temos nulos, vamos criar as features historicas
#Cria variáveis baseadas no histórico do cliente até a safra anterior.


def criar_features_historicas(df):
    df_feat = df.copy()
    df_feat = df.sort_values(by=['ID_CLIENTE', 'SAFRA_REF'])
    grouped = df_feat.groupby('ID_CLIENTE')
    
    

    # 1. Histórico de Bom Pagador: Contagem de pagamentos adiantados
    # Primeiro, criamos uma flag temporária para pagamentos adiantados (atraso < 0)

    df_feat['PAGOU_ADIANTADO'] = (df_feat['DIAS_DE_ATRASO'] < 0).astype(int)

    # Agora, calculamos a soma cumulativa de pagamentos adiantados e usamos shift(1) para pegar o histórico
    df_feat['HIST_PAGAMENTOS_ADIANTADOS'] = grouped['PAGOU_ADIANTADO'].cumsum().shift(1)

    # 2. Dias de Atraso Acumulados
    # Soma cumulativa dos dias de atraso e usa shift(1)
    df_feat['HIST_DIAS_ATRASO_ACUMULADO'] = grouped['DIAS_DE_ATRASO'].cumsum().shift(1)

    # 3. Dias de Atraso nos Últimos 3 registros
    # Usamos uma janela móvel (rolling window) de 3 períodos
    # O .reset_index() é necessário para realinhar o resultado com o DataFrame original
    soma_atraso_3m = grouped['DIAS_DE_ATRASO'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
    df_feat['HIST_DIAS_ATRASO_3M'] = soma_atraso_3m.shift(1)

    

    # 4. Total de Transações Anteriores
    # cumcount() conta as ocorrências anteriores dentro do grupo (começa em 0)
    df_feat['HIST_QTD_TRANSACOES'] = grouped.cumcount()
    
    #5. Valor medio pago
    df_feat['HIST_VALOR_MEDIO_PAGO'] = grouped['VALOR_A_PAGAR'].expanding().mean().reset_index(level=0, drop=True).shift(1)

    # 6. Flag de Inadimplência no Último periodo 
    df_feat['HIST_INADIMPLENTE_ULT_MES'] = grouped['INADIMPLENTE'].shift(1)

    # 7. Média de Dias de Atraso no Histórico
    df_feat['HIST_MEDIA_DIAS_ATRASO'] = (
    grouped['DIAS_DE_ATRASO']
    .expanding()
    .mean()
    .reset_index(level=0, drop=True)
    .shift(1)
)


    # O shift() cria valores nulos (NaN) para a primeira transação de cada cliente.
    colunas_hist = [col for col in df_feat.columns if 'HIST_' in col]
    df_feat[colunas_hist] = df_feat[colunas_hist].fillna(0)

    # Remover a coluna temporária
    df_feat = df_feat.drop(columns=['PAGOU_ADIANTADO'])
    
    return df_feat

data_pagamentos_hist_features = criar_features_historicas(data_pagamentos)



In [154]:
#Juntando as bases de dados para formar somente uma base de dados
join_data = (
    data_pagamentos_hist_features
    .merge(data_cadastro, on='ID_CLIENTE', how='left')
    .merge(data_hist, on=['ID_CLIENTE', 'SAFRA_REF'], how='left')
)

In [155]:
join_data.shape

(77367, 26)

In [156]:
join_data.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
       'DATA_VENCIMENTO', 'VALOR_A_PAGAR', 'TAXA', 'DIAS_ATE_PAGAMENTO',
       'INADIMPLENTE', 'DIAS_DE_ATRASO', 'PRAZO_PAGAMENTO_DIAS',
       'HIST_PAGAMENTOS_ADIANTADOS', 'HIST_DIAS_ATRASO_ACUMULADO',
       'HIST_DIAS_ATRASO_3M', 'HIST_QTD_TRANSACOES', 'HIST_VALOR_MEDIO_PAGO',
       'HIST_INADIMPLENTE_ULT_MES', 'HIST_MEDIA_DIAS_ATRASO', 'DATA_CADASTRO',
       'FLAG_PF', 'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE', 'CEP_2_DIG',
       'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS'],
      dtype='object')

In [157]:
join_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77367 entries, 0 to 77366
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ID_CLIENTE                  77367 non-null  int64         
 1   SAFRA_REF                   77367 non-null  datetime64[ns]
 2   DATA_EMISSAO_DOCUMENTO      77367 non-null  datetime64[ns]
 3   DATA_PAGAMENTO              77367 non-null  datetime64[ns]
 4   DATA_VENCIMENTO             77367 non-null  datetime64[ns]
 5   VALOR_A_PAGAR               77367 non-null  float64       
 6   TAXA                        77367 non-null  float64       
 7   DIAS_ATE_PAGAMENTO          77367 non-null  int64         
 8   INADIMPLENTE                77367 non-null  int64         
 9   DIAS_DE_ATRASO              77367 non-null  int64         
 10  PRAZO_PAGAMENTO_DIAS        77367 non-null  int64         
 11  HIST_PAGAMENTOS_ADIANTADOS  77367 non-null  float64   

In [158]:
join_data.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATE_PAGAMENTO,INADIMPLENTE,DIAS_DE_ATRASO,...,HIST_INADIMPLENTE_ULT_MES,HIST_MEDIA_DIAS_ATRASO,DATA_CADASTRO,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,8784237149961904,2018-08-01,2018-08-17,2018-09-04,2018-09-04,100616.10,5.99,18,0,0,...,0.00,0.00,2011-02-14,PJ,Comércio,HOTMAIL,PEQUENO,27,NaN,NaN
1,8784237149961904,2018-08-01,2018-08-22,2018-09-11,2018-09-11,89552.80,5.99,20,0,0,...,0.00,0.00,2011-02-14,PJ,Comércio,HOTMAIL,PEQUENO,27,NaN,NaN
2,8784237149961904,2018-08-01,2018-08-23,2018-09-10,2018-09-10,94062.80,5.99,18,0,0,...,0.00,0.00,2011-02-14,PJ,Comércio,HOTMAIL,PEQUENO,27,NaN,NaN
3,8784237149961904,2018-08-01,2018-08-23,2018-09-08,2018-09-10,102686.10,5.99,16,0,-2,...,0.00,0.00,2011-02-14,PJ,Comércio,HOTMAIL,PEQUENO,27,NaN,NaN
4,8784237149961904,2018-08-01,2018-08-24,2018-09-11,2018-09-11,51393.00,5.99,18,0,0,...,0.00,-0.50,2011-02-14,PJ,Comércio,HOTMAIL,PEQUENO,27,NaN,NaN


In [159]:
join_data = join_data.drop(columns=['DOMINIO_EMAIL'])
join_data.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
       'DATA_VENCIMENTO', 'VALOR_A_PAGAR', 'TAXA', 'DIAS_ATE_PAGAMENTO',
       'INADIMPLENTE', 'DIAS_DE_ATRASO', 'PRAZO_PAGAMENTO_DIAS',
       'HIST_PAGAMENTOS_ADIANTADOS', 'HIST_DIAS_ATRASO_ACUMULADO',
       'HIST_DIAS_ATRASO_3M', 'HIST_QTD_TRANSACOES', 'HIST_VALOR_MEDIO_PAGO',
       'HIST_INADIMPLENTE_ULT_MES', 'HIST_MEDIA_DIAS_ATRASO', 'DATA_CADASTRO',
       'FLAG_PF', 'SEGMENTO_INDUSTRIAL', 'PORTE', 'CEP_2_DIG',
       'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS'],
      dtype='object')

# Preparando a base completa para o modelo

## Tratando nulos

In [161]:
#Analisando os dados vamos preencher com base na mediana a renda do mes anterior e com base na media no
#nro de funcionarios

def preencher_valores_faltantes(df):
    df['RENDA_MES_ANTERIOR'] = df.groupby(['PORTE', 'SEGMENTO_INDUSTRIAL'])['RENDA_MES_ANTERIOR'].transform(
        lambda x: x.fillna(x.median())
    )

    df['NO_FUNCIONARIOS'] = df.groupby(['PORTE', 'SEGMENTO_INDUSTRIAL'])['NO_FUNCIONARIOS'].transform(
        lambda x: x.fillna(x.mean())
    )

    return df


join_data_filled = (
    join_data.copy()
    .pipe(preencher_valores_faltantes)
)

join_data_filled.isnull().sum()

ID_CLIENTE                    0
SAFRA_REF                     0
DATA_EMISSAO_DOCUMENTO        0
DATA_PAGAMENTO                0
DATA_VENCIMENTO               0
VALOR_A_PAGAR                 0
TAXA                          0
DIAS_ATE_PAGAMENTO            0
INADIMPLENTE                  0
DIAS_DE_ATRASO                0
PRAZO_PAGAMENTO_DIAS          0
HIST_PAGAMENTOS_ADIANTADOS    0
HIST_DIAS_ATRASO_ACUMULADO    0
HIST_DIAS_ATRASO_3M           0
HIST_QTD_TRANSACOES           0
HIST_VALOR_MEDIO_PAGO         0
HIST_INADIMPLENTE_ULT_MES     0
HIST_MEDIA_DIAS_ATRASO        0
DATA_CADASTRO                 0
FLAG_PF                       0
SEGMENTO_INDUSTRIAL           0
PORTE                         0
CEP_2_DIG                     0
RENDA_MES_ANTERIOR            0
NO_FUNCIONARIOS               0
dtype: int64

In [162]:
join_data_filled.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
       'DATA_VENCIMENTO', 'VALOR_A_PAGAR', 'TAXA', 'DIAS_ATE_PAGAMENTO',
       'INADIMPLENTE', 'DIAS_DE_ATRASO', 'PRAZO_PAGAMENTO_DIAS',
       'HIST_PAGAMENTOS_ADIANTADOS', 'HIST_DIAS_ATRASO_ACUMULADO',
       'HIST_DIAS_ATRASO_3M', 'HIST_QTD_TRANSACOES', 'HIST_VALOR_MEDIO_PAGO',
       'HIST_INADIMPLENTE_ULT_MES', 'HIST_MEDIA_DIAS_ATRASO', 'DATA_CADASTRO',
       'FLAG_PF', 'SEGMENTO_INDUSTRIAL', 'PORTE', 'CEP_2_DIG',
       'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS'],
      dtype='object')

In [163]:
data_objetivo.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_VENCIMENTO',
       'VALOR_A_PAGAR', 'TAXA'],
      dtype='object')

In [ ]:
#Criando variável de sazonalidade

join_data_filled['MES_EMISSAO'] = join_data_filled['DATA_EMISSAO_DOCUMENTO'].dt.month


## Lidar com categoricas

In [182]:
join_data_filled.shape

(77367, 26)

In [184]:
join_data_filled['CEP_2_DIG'].value_counts()

CEP_2_DIG
13    3886
35    2845
68    2603
89    2587
86    2135
      ... 
24     110
51      91
53      37
52      29
na       8
Name: count, Length: 90, dtype: int64

In [210]:
def map_regiao_cep(cep2):
    try:
        cep2 = int(cep2)
    except:
        return 'Desconhecido'

    if 1 <= cep2 <= 39:
        return 'Sudeste'
    elif cep2 in set(range(40, 50)) | set(range(60, 68)) | {67}:
        return 'Nordeste'
    elif cep2 in {68, 69, 77}:
        return 'Norte'
    elif cep2 in {70, 71, 72, 73, 74, 75, 76}:
        return 'Centro-Oeste'
    elif 80 <= cep2 <= 89:
        return 'Sul'
    else:
        return 'Desconhecido'


df_modelo_encoded = join_data_filled.copy()
df_modelo_encoded['REGIAO_CEP'] = df_modelo_encoded['CEP_2_DIG'].apply(map_regiao_cep)

# Confirme que a coluna foi criada
print(df_modelo_encoded['REGIAO_CEP'].value_counts(dropna=False))


REGIAO_CEP
Sudeste         32140
Sul             12788
Desconhecido    12342
Nordeste        11393
Centro-Oeste     5148
Norte            3556
Name: count, dtype: int64


In [211]:
# Remover CEP_2_DIG e aplicar one-hot
df_modelo_encoded.drop(columns='CEP_2_DIG', inplace=True)
df_modelo_encoded = pd.get_dummies(df_modelo_encoded, columns=['REGIAO_CEP'], drop_first=True)

In [212]:

# Label Encoding para PORTE (ordinals)
df_modelo_encoded['PORTE'] = df_modelo_encoded['PORTE'].astype('category').cat.codes

In [213]:
# One-Hot Encoding para SEGMENTO_INDUSTRIAL
colunas_onehot = ['SEGMENTO_INDUSTRIAL']
df_modelo_encoded = pd.get_dummies(df_modelo_encoded, columns=colunas_onehot, drop_first=True)

In [214]:

print("Colunas finais após encoding:")
print(df_modelo_encoded.shape)

Colunas finais após encoding:
(77367, 32)


In [215]:
# Salve a base final
df_modelo_encoded.to_parquet("../data/final/dados_modelo_pronto.parquet", index=False)
